In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import json

In [2]:
df = pd.read_csv("int20h-ds-test-dataset.csv")
df['event_created_date'] = df['event_created_date'].apply(lambda x: pd.Timestamp(x))
df['event_attributes'] = df['event_attributes'].apply(lambda x: json.loads(x))

events_df = pd.DataFrame( df['event_attributes'].tolist() )
events_col = events_df.columns
df[events_col] = events_df

df.head(5)

,userid,user_state,event_name,event_attributes,event_created_date,event_platform,device_manufacture,device_model,Make,Model,...,Type,Navigated From,Status,Transaction type,Premium Membership,Description,Google Map Installed,Selected Route Type,Transaction Type,Promo code
0,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Success,"{'Make': 'Dodge', 'Model': 'Caravan', 'Color':...",2022-01-16 17:03:04,android,samsung,SM-N975U,Dodge,Caravan,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,c95c777785faec8dd910d019d7278ebe,CA,Add Vehicle Break,{},2022-01-16 17:07:47,android,samsung,SM-N975U,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,f344be2d9a042b7444f3cc5279e38ef1,FL,Calculator View,{},2022-01-16 17:16:25,android,samsung,SM-G973U1,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,c95c777785faec8dd910d019d7278ebe,CA,Add Payment Method Success,"{'Payment Method': 'Credit', 'Tokenized Pay': ''}",2022-01-16 17:24:22,android,samsung,SM-N975U,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,e331ed81422d8fba55520a43a872e701,IL,Sign Up Success,{'Method': 'Apple'},2022-01-16 17:34:51,ios,Apple,"iPhone12,1",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
df['Start Date'].replace('', np.nan, inplace = True) #тут дати в різних форматах
df['End Date'].replace("", np.nan, inplace=True) #тут теж
df['Rental State'].replace("", np.nan, inplace=True) #тут штат, але їх дуже мало заповнених
df['Tokenized Pay'].replace("", np.nan, inplace=True) #тут або 'Apple Pay', "Google Pay", або np.nan
df['From'].replace("", np.nan, inplace=True)
df['Transaction type'].replace("", np.nan, inplace=True)

df.drop(columns=['Start Date','Rental state', 'Id', 'Transaction Type',"Description","Google Map Installed", "Selected Route Type", "Promo code"], inplace = True)
events_col = (events_col & df.columns)

/tmp/ipykernel_74652/2152467844.py:9: FutureWarning: Index.__and__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__and__.  Use index.intersection(other) instead.
  events_col = (events_col & df.columns)


In [4]:
df[events_col].notna().sum()

Make                  1923
Model                 1923
Color                 1923
Year                  1923
Temporary             1923
End Date               106
Payment Method        1372
Tokenized Pay          531
Method                3355
Rental State           102
Error Code             355
Error Message          355
Amount                 102
Required               547
Renewal Date          1021
Membership Type       1021
From                  1443
Screen                 498
Type                  2204
Navigated From        1609
Status                1230
Transaction type       172
Premium Membership    6116
dtype: int64

In [5]:
# ["Make", "Model", "Color", "Year", "Temporary"] - йдуть разом, event_name="Add Vehicle Success" (only)
# Start Date співпадає із event_created_date
# End Date є лише при Add Vehicle Success транзакції

# Payment Method - ['Debit', 'Credit', "Credit Card", "PayPal", "Venmo", "Google Pay"] виникає тільки при подіях "Add Payment Method Success", "Add Payment Method Failure"
# Tokenized Pay - ["Google Pay", "Apple Pay", np.nan] - виникає тільки при подіях "Add Payment Method Success", "Add Payment Method Failure"
# Method - ["Google", "Email", "Apple", "Facebook"]
# Error Code, Error Message - виникають тільки при івентах "Add Payment Method Failed", "Add Vehicle Failed"
# Id, Amount виникає тільки при події Transaction Refund (можна видалити)
# Required - виникає тільки при івенті "Email Confirmation Success"
# Renewal Date - тут дата, виникає тільки при подіях "Subscription Premium", "Subscription Premium Renew"
# Membership Type - тут два варіанти ['Monthly', 'Yearly'], в парі із Renewal Date
# From - виникає лише із івентом "Chat Conversation Opened". ["Menu", "Transaction Details", "Dashboard", "Deeplink"]
# Screen - виникає при події "Account Setup Profile Skip"

# Type - виникає при подіях "Account History Transaction Details", "Sign Out"
# Navigated From - виникає при різних івентах, але має лише два значення (присутній/відсутній)
# Status - виникає при події "Account History Transaction Details"

# Transaction type - виникає при події "Chat Conversation Opened"
# Premium Membership - True/False/np.nan
# Promo code - виникає при події "Add Payment Method Success", можна просто фіксувати наявність